# RSK Parameters

This workbook explores how the magnitudes of the different parameters affect the estimated means.  We observe that when the `alpha` process can be estimated with high confidence (i.e. `Q` is small) then RSK substantially outperforms the naive mean, even when `sigma` is huge. Essentially small `Q` dampens out the noise and so that the true `alpha` process dominates the estimation.

In [71]:
import numpy as np
import scipy as sp
from rsk.rsk import RSK
from rsk.panel import PanelSeries

# parameters
N,T = 30, 10  # sample size, periods
a0 = 1        # initial alpha 
s = 1e3       # variance scale
f = 1.01      # transition scale
q = 1e-2      # alpha var scale

####################################
y = np.zeros((N, T))
alpha, alpha_true  = np.zeros(T), np.zeros(T)
alpha[0] = a0
alpha_true[0]=a0

sigma = s*sp.eye(1)
Q = q*np.eye(1)
F,Z = f*np.eye(1), np.eye(1)

y[:,0] = alpha[0] + np.random.normal(0, sigma, size=(N,))
for i in range(1,T):
    alpha[i] = f*alpha[i-1] + np.random.normal(0, Q)
    y[:,i] = alpha[i] + np.random.normal(0, sigma, size=(N,))
    alpha_true[i] = F*alpha_true[i-1]

# build panel series
rows = []    
for i,col in enumerate(y.transpose()):
    for entry in col:
        rows.append([i,"A",entry])
ps = PanelSeries.from_list(rows)

###########################################
# compute means
naive_means = y.mean(axis=0)

a0 = a0*np.eye(1)
rsk = RSK(F,Z)
fitted_means_smooth = [x[0][0] for x in rsk.fit(ps, a0, Q, Q, smooth=True)]
fitted_means = [x[0][0] for x in rsk.fit(ps, a0, Q, Q, smooth=False)]

###########################################
print("mu\t\trsk_smooth\trsk\t\tnaive")
for a,w,x,y in zip(alpha_true, fitted_means_smooth, fitted_means, naive_means):
    print("%.8f\t%.8f\t%.8f\t%.8f" % (a,w,x,y))

mu		rsk_smooth	rsk		naive
1.00000000	1.00981479	1.00985503	-266.76760527
1.01000000	1.01989322	1.02004261	109.55586757
1.02010000	1.03004382	1.03061807	340.50671640
1.03030100	1.04020634	1.04060471	-228.02541965
1.04060401	1.05053262	1.05067201	-199.27471336
1.05101005	1.06101607	1.06090846	-134.55041069
1.06152015	1.07164056	1.07145105	-27.82914633
1.07213535	1.08237882	1.08190748	-97.60002055
1.08285671	1.09325043	1.09280380	27.40484954
1.09368527	1.10422330	1.10422330	151.72964472
